## ROC extraction

15/06/16 by greghor


on commence par regarder les documents ou l on observe le mot center
utilise les fichiers .html
integre corrections typographiques pour coller à la solution
integre ROC à partir des currencies

In [6]:
# SYSTEM
from os import listdir
from PyCommonFun import *

#NLTK
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from nltk.metrics import *

#GENERAL
import numpy as np
import pandas as pd
import pickle
import string

# data from Albert
from code.dataExtract import data
import code.dataExtract as de
from code import Text_reader as tr
# Read the data
d = data(folder='../../train/') # training set with all the labels
dataPath='../../train/'



In [7]:
# fonctions used in the notebook

def getContext(s,pattern,context=list(),width=100):
    
    
    loc=s.find(pattern)
    
    if loc == -1:
        return context
    elif loc-width < 0:
        context+=s[:loc+width]
        return getContext(s[loc+len(pattern):],pattern,context,width)
    elif loc + width > len(s):
        context+=s[loc-width:]
        return getContext(s[loc+len(pattern):],pattern,context,width)
    else:
        context+=s[loc-width:loc+width]
        return getContext(s[loc+len(pattern):],pattern,context,width)


    
def useSolutionTypo(s):
    s=s.lower()
    if s=="sao paulo":
        s="sao-paulo"
    elif s=='target 1' or s =='target 2' or s == 'target' or s=='target1':
        s='target holidays'
    return s



def document_from_hmtl(doc_path):
    with open(doc_path) as html_file:
        full_text = html_file.read()
        body = re.findall(r'<body[^>]*?>(.*?)</body>', full_text)
        cleantext = re.sub('<\/?span[^>]*>', ' ', body[0])
        clean_text = re.sub('<.*?>', ' ', cleantext)
        clean_text = clean_text.lower()
        clean_text = clean_text.lower()
        clean_text = re.sub('0.01', '1', clean_text)
        clean_text = re.sub(',', '', clean_text)
        clean_text = re.sub(r'[^a-z0-9]', ' ', clean_text)
        clean_text = re.sub('\s+', ' ', clean_text)
        return clean_text

def removeUnwantedCity(city, unwantedCity):
    #
    assert isinstance(city,list) or isinstance(city,np.ndarray), "city should be an np.array or a list"
    assert isinstance(unwantedCity,list),  "unwanted city should be a list"
        
    for gi in range(len(city)):
        for unwanted in unwantedCity:
            if '|' + unwanted in city[gi]:
                city[gi]=city[gi].replace('|'+ unwanted,'')
            elif unwanted + '|' in city[gi]:
                city[gi]=city[gi].replace(unwanted + '|','')
            elif unwanted in city[gi]:
                city[gi]=city[gi].replace(unwanted,'')
    return city
                

In [8]:
# get the solution
ROCTmp=pd.read_csv('ROC_fullList.csv',delimiter=";")
cities_groundTruth=np.load('../currencyExtraction/predicted_opCurr_dict.npy').item()
ROC_list=['target 1','target 2','target'] # add target 1 and target 2
# lowerise ROC and remove ponctuation
for roc in ROCTmp['CIT_NM'].values:
    roc=str(roc).lower().translate(None, string.punctuation)
    ROC_list.append(roc.rstrip())

In [20]:
# link roc and operationnal curency
opCurrency=np.load('../currencyExtraction/predicted_opCurr_dict.npy').item()
opCurr_df=pd.DataFrame(opCurrency.items(),columns=['ISIN','OpCurrency'])
link_roc_opCurr=pd.read_csv('currency_ROC_fullList.csv',delimiter=";")
# merge with labels on operationnal currency
roc_fromCurrency=pd.merge(left=link_roc_opCurr,right=opCurr_df,left_on='Currency Code', right_on='OpCurrency')
# keep only currency code and default ROC columns
col=["ISIN","OpCurrency","Default ROC",]
roc_fromCurrency=roc_fromCurrency[col]
roc_fromCurrency.rename(columns={'Default ROC':'currency_ROC'},inplace=True)
len(roc_fromCurrency)

426

In [13]:
gf=0


context_pat=dict()
text=dict()
htmlFiles=tr.get_files(dataPath + '/html')



for isin in d.docid.keys()[:10]:
        
    text[isin]=document_from_hmtl(htmlFiles[d.docid[isin][0]])
    context_pat[isin]=dict()

    pattern=["Business Day Centre","Financial Centre","Business Centre","Currency Business Day",'Business Day']
    
    for pat in pattern:
        context_pat[isin][pat]=getContext(text[isin],pat.lower(),'',150)
        # il faut impérativement passer, un troisieme argument vide pour que le code 
        # fonctionne mais je ne comprends pas bien pourquoi
            
            
            
    gf+=1
    print float(gf)/len(d.docid.keys())
       


0.000183284457478
0.000366568914956
0.000549853372434
0.000733137829912
0.00091642228739
0.00109970674487
0.00128299120235
0.00146627565982
0.0016495601173
0.00183284457478


In [15]:
# try to extract the ROC 

extractedROC_Raw_dict=dict() # contains only cities extracted from the documents 


for isin in context_pat.keys():
    
    extractedROC_Raw_dict[isin]=dict()
    
    currencyROCTmp=roc_fromCurrency['currency_ROC'].loc[roc_fromCurrency['ISIN']==isin].values[0].lower()
    currencyROCTmp=useSolutionTypo(currencyROCTmp)
    
    extractedROC_Raw_dict[isin]['currencyROC']=set([currencyROCTmp])
    
    for pat in context_pat[isin].keys():
    
        extractedROC_Raw_dict[isin][pat]=set()
        
        for roc in ROC_list:
            
            if roc in ''.join(context_pat[isin][pat]):
                 extractedROC_Raw_dict[isin][pat].add(useSolutionTypo(roc))
                    
               
                
                 

IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:
isin in roc_fromCurrency['ISIN']

False

In [ ]:
extractedROC_Raw_dict['XS1267246244'].keys()


#### retenir uniquement une solution sur extractedROC_Raw_dict avec l ordering :
["Business Day Centre","Financial Centre","Business Centre","Currency Business Day",'Business Day']



In [ ]:
# on compare les sets par ordre de priorites des patterns et on retient le premier qui est non vide
extractedROC_sol=dict()
for fid in extractedROC_Raw_dict.keys():
    gpat=0
    extractedROC_sol[fid]=dict()
    while gpat <= 3 and len(extractedROC_Raw_dict[fid][pattern[gpat]])==0:
        gpat+=1
    extractedROC_sol[fid]['cities']=extractedROC_Raw_dict[fid][pattern[gpat]]
    extractedROC_sol[fid]['pattern']=pattern[gpat]
    # always add currencies ROC
    extractedROC_sol[fid]['cities']=extractedROC_sol[fid]['cities'].union(extractedROC_Raw_dict[fid]['currencyROC'])




In [ ]:
#remove spurious cities that do not appear in the ground truth (dangerous)
# but the method seems to work
citiesGT=pd.read_csv('/Users/greghor/Documents/taf/dataScience/Euroclear_hackathon/notebook/git_dieuhack/data_0/cities.csv')
setUniCitiesGT=set(x.lower() for x in citiesGT['City.Name'].values)

setROCRefList=set()
for x in ROC_list:
    setROCRefList.update(x.lower().split('|'))
spuriousCities=setROCRefList-setUniCitiesGT

for fid in extractedROC_sol.keys():
    extractedROC_sol[fid]['cities']=extractedROC_sol[fid]['cities']-spuriousCities;

In [ ]:
def addPip_betweenCities(citiesSet):
    cityString=str()
    for cit in citiesSet:
        if len(cityString)==0:
            cityString=cit
        else:
            cityString=cityString + '|' + cit
    return cityString

# turn to dataset and pipe the cities
extractedROC_df=pd.DataFrame()
col=['isin','myCities','pattern']
for col in col:
    extractedROC_df[col]=''
    
for isin in extractedROC_sol.keys():
    newRow=[isin]
    newRow.append(addPip_betweenCities(extractedROC_sol[isin]['cities']).upper())
    newRow.append(extractedROC_sol[isin]['pattern'])
    extractedROC_df.loc[len(extractedROC_df)]=newRow

In [ ]:
extractedROC_df

In [ ]:
# merge with citiesGreg to check how many results are OK

citiesGreg=pd.read_csv('citiesGreg.csv')
joinedCities=pd.merge(citiesGreg,extractedROC_df,on='isin')

In [ ]:
# check the compliance between ground truth and my solution
indCheck=[]
gval=0
for multiCity in joinedCities['myCities'].values:
    indCheckTmp=True
    for city in multiCity.split('|'): # check that all extracted cities are in the ground Truth
        indCheckTmp=indCheckTmp and (city in  joinedCities['citiesName'].values[gval])

    # check that ground truth do not content additionnal city
    if len(joinedCities['citiesName'].values[gval]) == len(multiCity):
        indCheckTmp=indCheckTmp and True
    else:
        indCheckTmp=indCheckTmp and False
    indCheck.append(indCheckTmp)
    gval+=1
joinedCities["check"  ]=indCheck

# reorder the columns for clarity
col=['fileId',
 'isin',
 'citiesName', 'myCities','check',
 'pattern']


In [ ]:
check_extractedCities=joinedCities[col]
check_extractedCities.to_csv('check_extractedCities.csv')

In [ ]:
len(PyFind(check_extractedCities['check'].values, lambda x: x==True))/np.float(len(check_extractedCities['citiesName'].values))

In [ ]:
extractedROC_sol['XS1266607586']